In [1]:
# Importer les bibliothèques nécessaires
import pickle
import os
import json
import torch
from torch.utils.data import Dataset, DataLoader,random_split
import cv2
import numpy as np
import torchvision.transforms as transforms
from PIL import Image
import torch.nn as nn
from torchvision.models import resnet50,ResNet50_Weights
from transformers import BertModel, BertTokenizer, BartForConditionalGeneration, BartTokenizer
from transformers import AdamW


In [2]:
#Préparation du dataset d'entrainnement en un seul fichier pickle

In [ ]:

# Définir les chemins vers les images et les fichiers JSON
images_dir = "dataset VQA/train2014/train2014"
questions_file = "dataset VQA/v2_Questions_Train_mscoco/v2_OpenEnded_mscoco_train2014_questions.json"
annotations_file = "dataset VQA/v2_Annotations_Train_mscoco/v2_mscoco_train2014_annotations.json"
output_pickle_file = "dataset_v3/vqa_dataset_part.pkl"  # Modifié pour créer plusieurs fichiers

# Charger les données des questions et annotations depuis les fichiers JSON
try:
    with open(questions_file, 'r', encoding='utf-8') as qfile:
        questions_data = json.load(qfile)

    with open(annotations_file, 'r', encoding='utf-8') as afile:
        annotations_data = json.load(afile)
except json.JSONDecodeError as e:
    print(f"Erreur de décodage JSON: {e}")
    questions_data = {}
    annotations_data = {}
except Exception as e:
    print(f"Une erreur est survenue: {e}")
    questions_data = {}
    annotations_data = {}

# Indexer les questions et annotations par image_id pour un accès rapide
questions_index = {}
for question in questions_data.get("questions", []):
    image_id = question["image_id"]
    if image_id not in questions_index:
        questions_index[image_id] = []
    questions_index[image_id].append(question["question"])

annotations_index = {}
for annotation in annotations_data.get("annotations", []):
    image_id = annotation["image_id"]
    if image_id not in annotations_index:
        annotations_index[image_id] = []
    annotations_index[image_id].append(annotation["multiple_choice_answer"])

# Diviser les images en 10 lots
total_images = len(questions_index)
batch_size = total_images // 10

# Fonction pour charger un lot d'images et les sauver dans un fichier pickle
def load_and_save_batch(start_idx, end_idx, batch_number):
    vqa_dataset_batch = []  # Liste au lieu de dictionnaire
    
    for idx, (image_id, questions) in enumerate(list(questions_index.items())[start_idx:end_idx], start=start_idx + 1):
        # Formater le nom de l'image pour correspondre aux fichiers d'image
        formatted_image_id = f"COCO_train2014_{image_id:012d}"
        image_path = os.path.join(images_dir, f"{formatted_image_id}.jpg")
        
        # Vérifier si l'image existe
        if not os.path.exists(image_path):
            print(f"Image non trouvée pour l'ID {formatted_image_id}")
            continue
        
        # Construire les paires question-réponse et les ajouter sous forme de tuple [image_path, question, answer]
        for question, answer in zip(questions, annotations_index.get(image_id, [])):
            vqa_dataset_batch.append([image_path, question, answer])
        
        # Afficher le pourcentage de chargement du lot
        progress = (idx / total_images) * 100
        print(f"Chargement du lot {batch_number} : {progress:.2f}% ({idx}/{total_images})")
    
    # Sauvegarder le lot dans un fichier pickle
    with open(output_pickle_file.format(batch_number), 'wb') as pfile:
        pickle.dump(vqa_dataset_batch, pfile)
        print(f"Lot {batch_number} sauvegardé dans {output_pickle_file.format(batch_number)}")

# Charger et sauver les 10 lots pour faciliter le traitement
for batch_number in range(10):
    start_idx = batch_number * batch_size
    # Si c'est le dernier lot, on va jusqu'à la fin des données
    end_idx = (batch_number + 1) * batch_size if batch_number < 9 else total_images
    load_and_save_batch(start_idx, end_idx, batch_number + 1)

print("Tous les lots ont été traités et sauvegardés.")


In [ ]:
# Chemin du dossier contenant les fichiers de lots
batch_files_dir = "dataset_v3"
# Nom des fichiers de lot
batch_file_template = "vqa_dataset_part_{}.pkl"
# Nom du fichier de sortie unique
output_file = os.path.join(batch_files_dir, "vqa_dataset_full.pkl")

# Liste pour stocker tous les lots fusionnés
full_dataset = []

# Charger chaque fichier de lot et l'ajouter à la liste complète
for batch_number in range(10):
    batch_file = os.path.join(batch_files_dir, batch_file_template.format(batch_number + 1))
    try:
        with open(batch_file, 'rb') as pfile:
            batch_data = pickle.load(pfile)
            full_dataset.extend(batch_data)  # Fusionner les lots en ajoutant chaque élément de batch_data
            print(f"Lot {batch_number + 1} fusionné avec succès.")
    except Exception as e:
        print(f"Erreur lors du chargement du lot {batch_number + 1}: {e}")

# Sauvegarder le dataset complet dans un fichier pickle unique (fusion)
with open(output_file, 'wb') as outfile:
    pickle.dump(full_dataset, outfile)
    print(f"Dataset complet sauvegardé dans {output_file}")


In [3]:
#Préparation données validation en un seul fichier pickle

In [ ]:

# Définir les chemins vers les images et les fichiers JSON
images_dir = "dataset VQA/val2014/val2014"
questions_file = "dataset VQA/v2_Questions_Val_mscoco/v2_OpenEnded_mscoco_val2014_questions.json"
annotations_file = "dataset VQA/v2_Annotations_Val_mscoco/v2_mscoco_val2014_annotations.json"
output_pickle_file = "dataset_v3/vqa_dataset_val_part_{}.pkl"  

# Charger les données des questions et annotations depuis les fichiers JSON
try:
    with open(questions_file, 'r', encoding='utf-8') as qfile:
        questions_data = json.load(qfile)

    with open(annotations_file, 'r', encoding='utf-8') as afile:
        annotations_data = json.load(afile)
except json.JSONDecodeError as e:
    print(f"Erreur de décodage JSON: {e}")
    questions_data = {}
    annotations_data = {}
except Exception as e:
    print(f"Une erreur est survenue: {e}")
    questions_data = {}
    annotations_data = {}

# Indexer les questions et annotations par image_id pour un accès rapide
questions_index = {}
for question in questions_data.get("questions", []):
    image_id = question["image_id"]
    if image_id not in questions_index:
        questions_index[image_id] = []
    questions_index[image_id].append(question["question"])

annotations_index = {}
for annotation in annotations_data.get("annotations", []):
    image_id = annotation["image_id"]
    if image_id not in annotations_index:
        annotations_index[image_id] = []
    annotations_index[image_id].append(annotation["multiple_choice_answer"])

# Diviser les images en 10 lots pour faciliter le traitement
total_images = len(questions_index)
batch_size = total_images // 10

# Fonction pour charger un lot d'images et les sauver dans un fichier pickle
def load_and_save_batch(start_idx, end_idx, batch_number):
    vqa_dataset_batch = []  # Liste au lieu de dictionnaire
    
    for idx, (image_id, questions) in enumerate(list(questions_index.items())[start_idx:end_idx], start=start_idx + 1):
        # Formater le nom de l'image pour correspondre aux fichiers d'image
        formatted_image_id = f"COCO_val2014_{image_id:012d}"
        image_path = os.path.join(images_dir, f"{formatted_image_id}.jpg")
        
        # Vérifier si l'image existe
        if not os.path.exists(image_path):
            print(f"Image non trouvée pour l'ID {formatted_image_id}")
            continue
        
        # Construire les paires question-réponse et les ajouter sous forme de tuple [image_path, question, answer]
        for question, answer in zip(questions, annotations_index.get(image_id, [])):
            vqa_dataset_batch.append([image_path, question, answer])
        
        # Afficher le pourcentage de chargement du lot
        progress = (idx / total_images) * 100
        print(f"Chargement du lot {batch_number} : {progress:.2f}% ({idx}/{total_images})")
    
    # Sauvegarder le lot dans un fichier pickle
    with open(output_pickle_file.format(batch_number), 'wb') as pfile:
        pickle.dump(vqa_dataset_batch, pfile)
        print(f"Lot {batch_number} sauvegardé dans {output_pickle_file.format(batch_number)}")

# Charger et sauver les 10 lots
for batch_number in range(10):
    start_idx = batch_number * batch_size
    # Si c'est le dernier lot, on va jusqu'à la fin des données
    end_idx = (batch_number + 1) * batch_size if batch_number < 9 else total_images
    load_and_save_batch(start_idx, end_idx, batch_number + 1)

print("Tous les lots ont été traités et sauvegardés.")


In [ ]:
# Chemin du dossier contenant les fichiers de lots
batch_files_dir = "dataset_v3"
# Nom des fichiers de lot
batch_file_template = "vqa_dataset_val_part_{}.pkl"
# Nom du fichier de sortie unique
output_file = os.path.join(batch_files_dir, "vqa_dataset_val_full.pkl")

# Liste pour stocker tous les lots fusionnés
full_dataset = []

# Charger chaque fichier de lot et l'ajouter à la liste complète
for batch_number in range(10):
    batch_file = os.path.join(batch_files_dir, batch_file_template.format(batch_number + 1))
    try:
        with open(batch_file, 'rb') as pfile:
            batch_data = pickle.load(pfile)
            full_dataset.extend(batch_data)  # Fusionner les lots en ajoutant chaque élément de batch_data
            print(f"Lot {batch_number + 1} fusionné avec succès.")
    except Exception as e:
        print(f"Erreur lors du chargement du lot {batch_number + 1}: {e}")

# Sauvegarder le dataset complet dans un fichier pickle unique
with open(output_file, 'wb') as outfile:
    pickle.dump(full_dataset, outfile)
    print(f"Dataset complet sauvegardé dans {output_file}")
